In [57]:
import pycosat
import sympy
from sympy.logic import simplify_logic
from sympy.logic.inference import satisfiable
from sympy.parsing.sympy_parser import parse_expr, standard_transformations, implicit_multiplication_application
import itertools
import pandas as pd

In [362]:
def get_cnf(formula_str):
    formula = sympy.sympify(formula_str)
    cnf_formula = simplify_logic(formula, form='cnf')
    return cnf_formula


def evaluate_expression(expression, variable_values):
    expression_context = dict(zip(variable_values.keys(), variable_values.values()))
    # Replace 'and' with '&' and 'or' with '|'
    expression = expression.replace("&", " and ")
    expression = expression.replace("|", " or ")
    expression = expression.replace("~", " not ")
    while '>>' in expression:
        eq_index = expression.index(">>")
        
        left_par_index = expression[:eq_index].rfind('(')
        right_par_index = eq_index + expression[eq_index:].index(')')
        eq_substr = expression[left_par_index:right_par_index+1]
        #print(eq_substr)
        
        
        
        temp_str = eq_substr[0] + " not " +  eq_substr[1:]
        #print(eq_substr[-2:1:-1])
        
        temp_str = temp_str.replace('>>',' or ')
       
        #print(temp_str)

        expression = expression.replace(eq_substr, temp_str)
    return eval(expression, {}, expression_context)

def generate_truth_table(expression):
    variable_names = sorted(set([char for char in expression if char.isalpha() and  char.isupper() ]))
    combinations = list(itertools.product([False, True], repeat=len(variable_names)))
    
    
    truth_table_data = []
    for combination in combinations:
        variable_values = dict(zip(variable_names, combination))
        result = int(evaluate_expression(expression, variable_values))
        truth_table_data.append(combination + (result,))
    
    columns = variable_names + [expression]
    truth_table_df = pd.DataFrame(truth_table_data, columns=columns)
    print('Truth Table:')
    display(truth_table_df)


def replace_equivalence(formula_str):
    while "==" in formula_str:
        eq_index = formula_str.index("==")
        
        left_par_index = formula_str[:eq_index].rfind('(')
        right_par_index = eq_index + formula_str[eq_index:].index(')')
        eq_substr = formula_str[left_par_index:right_par_index+1]
        #print(eq_substr)
        
        
        
        rev_str = eq_substr[0] + eq_substr[-2:0:-1] + eq_substr[-1]
        #print(eq_substr[-2:1:-1])
        
        temp_str = (eq_substr + ' & ' + rev_str).replace('==','>>')
       
        #print(temp_str)

        formula_str = formula_str.replace(eq_substr, temp_str)

    return formula_str

def clause_to_pycosat(clause, symbol_to_int):
    pycosat_clause = []
    for literal in clause.args:
        if isinstance(literal, sympy.Not):
            pycosat_clause.append(-symbol_to_int[literal.args[0]])
        else:
            pycosat_clause.append(symbol_to_int[literal])
    return pycosat_clause

def convert_cnf_to_clauses(cnf_formula):
    if cnf_formula in (sympy.true, sympy.false):
        return [], {}

    symbols = list(cnf_formula.atoms(sympy.Symbol))
    symbol_to_int = {symbol: i + 1 for i, symbol in enumerate(symbols)}
    clauses = []

    if isinstance(cnf_formula, sympy.And):
        for clause in cnf_formula.args:
            if isinstance(clause, sympy.Or):
                pycosat_clause = clause_to_pycosat(clause, symbol_to_int)
            elif isinstance(clause, sympy.Not):
                pycosat_clause = [-symbol_to_int[clause.args[0]]]
            else:
                pycosat_clause = [symbol_to_int[clause]]
            clauses.append(pycosat_clause)
    else:
        if isinstance(cnf_formula, sympy.Or):
            pycosat_clause = clause_to_pycosat(cnf_formula, symbol_to_int)
        elif isinstance(cnf_formula, sympy.Not):
            pycosat_clause = [-symbol_to_int[cnf_formula.args[0]]]
        else:
            pycosat_clause = [symbol_to_int[cnf_formula]]
        clauses.append(pycosat_clause)

    return clauses, symbol_to_int


def find_models(formula_str):
    generate_truth_table(formula_str)
    formula_str = replace_equivalence(formula_str)
    formula = parse_expr(formula_str)
    cnf_formula = get_cnf(formula)
    
    

    if cnf_formula == sympy.false:
        print("The formula is unsatisfiable. There are no models.")
        return

    if cnf_formula == sympy.true:
        print("The formula is a tautology. All models satisfy the formula.")
        return

    clauses, symbol_to_int = convert_cnf_to_clauses(cnf_formula)
    models = list(pycosat.itersolve(clauses))

    if models:
        print("Models:")
        for model in models:
            print({symbol: (symbol_to_int[symbol] in model) for symbol in symbol_to_int.keys()})
    else:
        print("The formula is unsatisfiable. There are no models.")



In [363]:
find_models(formula_str = "(~A | B) & A & ~B & C & D")

Truth Table:


,A,B,C,D,(~A | B) & A & ~B & C & D
0,False,False,False,False,0
1,False,False,False,True,0
2,False,False,True,False,0
3,False,False,True,True,0
4,False,True,False,False,0
5,False,True,False,True,0
6,False,True,True,False,0
7,False,True,True,True,0
8,True,False,False,False,0
9,True,False,False,True,0


The formula is unsatisfiable. There are no models.


In [364]:
find_models(formula_str = "(A | B) & A & ~B & C & D")

Truth Table:


,A,B,C,D,(A | B) & A & ~B & C & D
0,False,False,False,False,0
1,False,False,False,True,0
2,False,False,True,False,0
3,False,False,True,True,0
4,False,True,False,False,0
5,False,True,False,True,0
6,False,True,True,False,0
7,False,True,True,True,0
8,True,False,False,False,0
9,True,False,False,True,0


Models:
{D: True, A: True, B: False, C: True}


In [365]:
find_models(formula_str = "(A | B) & A & ~B & ~C & D")

Truth Table:


,A,B,C,D,(A | B) & A & ~B & ~C & D
0,False,False,False,False,0
1,False,False,False,True,0
2,False,False,True,False,0
3,False,False,True,True,0
4,False,True,False,False,0
5,False,True,False,True,0
6,False,True,True,False,0
7,False,True,True,True,0
8,True,False,False,False,0
9,True,False,False,True,1


Models:
{D: True, C: False, B: False, A: True}


In [367]:
find_models(formula_str = "(A >> B)")

Truth Table:


,A,B,(A >> B)
0,False,False,1
1,False,True,1
2,True,False,0
3,True,True,1


Models:
{A: False, B: True}
{A: False, B: False}
{A: True, B: True}


In [368]:
find_models(formula_str = "(A == B)")   # ==: bidieectional

Truth Table:


,A,B,(A == B)
0,False,False,1
1,False,True,0
2,True,False,0
3,True,True,1


Models:
{A: False, B: False}
{A: True, B: True}


In [369]:
find_models(formula_str = "(A >> B) & A & ~B & ~C & D")

Truth Table:


,A,B,C,D,(A >> B) & A & ~B & ~C & D
0,False,False,False,False,0
1,False,False,False,True,0
2,False,False,True,False,0
3,False,False,True,True,0
4,False,True,False,False,0
5,False,True,False,True,0
6,False,True,True,False,0
7,False,True,True,True,0
8,True,False,False,False,0
9,True,False,False,True,0


The formula is unsatisfiable. There are no models.


In [370]:
find_models(formula_str = "(A == B) & A & ~B & ~C & D")

Truth Table:


,A,B,C,D,(A == B) & A & ~B & ~C & D
0,False,False,False,False,0
1,False,False,False,True,0
2,False,False,True,False,0
3,False,False,True,True,0
4,False,True,False,False,0
5,False,True,False,True,0
6,False,True,True,False,0
7,False,True,True,True,0
8,True,False,False,False,0
9,True,False,False,True,0


The formula is unsatisfiable. There are no models.
